<a href="https://colab.research.google.com/github/stephyi/Impact-of-Covid-19-on-Food-and-Air-Quality-in-Africa/blob/stephany/srapper_BlueBird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# -*- coding: utf-8 -*-
"""blueberry_pull_tweets.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1ynJvXI5vMYmu-fnNABCAQpL_GVuP41J_
"""



from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/Impact-of-Covid-19-on-Food-and-Air-Quality-in-Africa'

import pandas as pd
import numpy as np
import os
import re
import logging
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from datetime import datetime, date, time, timedelta
import tweepy
from tweepy import OAuthHandler, API, Cursor, Stream
from tweepy.streaming import StreamListener
from nltk.corpus import stopwords
from textblob import TextBlob
from collections import Counter
import sys
!pip install preprocessor
!pip install tweet-preprocessor
import preprocessor as p
!pip install GetOldTweets3 as got
import GetOldTweets3 as got
from GetOldTweets3.manager import TweetCriteria
!pip install bluebird
from bluebird import BlueBird



class dataAcq():
    def __init__(self, cols=None, auth=None):
        if cols is not None:
            self.cols = cols
        else:
            self.cols = ['id', 'created_at', 'original_text', 'clean_text',
                         'polarity', 'subjectivity', 'favorite_count', 'retweet_count',
                         'possibly_sensitive', 'hashtags', 'user_mentions', 'place']

    def cleanTweets(self, twitterText):
        tweet = p.clean(twitterText)

        # Happy Emoticons
        emoticonsHappy = set([
            ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
            ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
            '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
            'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
            '<3'])

        # Sad Emoticons
        emoticonsSad = set([
            ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
            ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
            ':c', ':{', '>:\\', ';('])

        # Emoji patterns
        emojiPattern = re.compile("[" u"\U0001F600-\U0001F64F"  # emoticons
                                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                      u"\U00002702-\U000027B0"
                                      u"\U000024C2-\U0001F251" "]+", flags=re.UNICODE)

        # Combine the sad and happy emoticons
        emoticons = emoticonsHappy.union(emoticonsSad)

        # tokenize the words in the tweet and get the stop words
        stopWords = set(stopwords.words('english'))
        wordTokens = nltk.word_tokenize(tweet)

        # after tweepy preprocessing the colon symbol left remain after
        # removing mentions
        tweet = re.sub(r':', '', tweet)
        tweet = re.sub(r'‚Ä¶', '', tweet)

        # replace consecutive non-ASCII characters with a space
        tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)

        # remove emojis from tweet
        tweet = emojiPattern.sub(r'', tweet)

        # looping through conditions
        filteredTweet = []
        for w in wordTokens:
            # check tokens against stop words , emoticons and punctuations
            if w not in stopWords and w not in emoticons and w not in string.punctuation:
                filteredTweet.append(w)

        return ' '.join(filteredTweet)

    def getTweets(self, location, hashtag, startDate,endDate):
        
        df = pd.DataFrame(columns=self.cols)

        query = {
          'fields': [{'items': [hashtag]}],
          'near': (location, 40),
          'since': startDate,
          'until': endDate
        }
            
        for status in BlueBird().search(query):
            newEntry = []
            # status = status._json

            # if this tweet is a retweet update retweet count
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]

                cond1 = status['favorite_count'] != df.at[i, 'favorite_count']
                cond2 = status['retweet_count'] != df.at[i, 'retweet_count']
                if cond1 or cond2:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']

                continue

            # clean the tweet
            cleanText = self.cleanTweets(status['full_text'])

            # Calculate the Sentiment
            blob = TextBlob(cleanText)
            polarity = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity

            newEntry += [status['id'], status['created_at'], status['full_text'], cleanText,
                        polarity, subjectivity, status['favorite_count'], status['retweet_count']]

            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None

            newEntry.append(is_sensitive)

            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            newEntry.append(hashtags)  # append the hashtags

            #
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            newEntry.append(mentions)  # append the user mentions

            try:
                location = status['place']['full_name']
            except TypeError:
                location = ''
            newEntry.append(location)

            # now append a row to the dataframe
            single_tweet_df = pd.DataFrame([newEntry], columns=self.cols)
            df = df.append(single_tweet_df, ignore_index=True)

        return df

hashtags =  ["#AirPollution",'#stopairpollution','#NO2Coal','EndCoal','AQI','#airqualityindex',"#airquality","#cleanair",
          "#airpollutant","#freshair", "Particulates","#pm2_5","#emissions",'#AIRS',
            '#airpollutionawareness', '#airpollutioncontrol','#airparticles', '#smog','#Aerosals','#smog', "#blueskychallenge", "#emissions", "#hvac"] 
      
locations = ["Lagos, Nigeria", "Cape Town, South Africa", "Johannesburg, South Africa", "Accra, Ghana", "Nairobi, Kenya", "Mombasa, Kenya", 
            "Kigali, Rwanda", "Kampala, Uganda"]

acquireTweets = dataAcq()
frames = []

for location in locations:
  df = []
  for hashtag in hashtags:
    tweets = acquireTweets.getTweets(location=location,hashtag=hashtag,startDate="2020-04-01",endDate="2020-07-31")
    df.append(tweets)
  conc_df = pd.concat(df, ignore_index=True)
  print(conc_df.shape)
  frames.append(conc_df)
final_df = pd.concat(frames, ignore_index=True)
print(final_df.shape)

final_df

final_df.to_csv('data.csv', index=False, encoding="utf-8")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[Test URL] https://twitter.com/search?f=tweets&vertical=default&q=%23AirPollution%20since%3A2020-04-01%20until%3A2020-07-31%20near%3A%22Lagos%2C%20Nigeria%22%20within%3A40mi
https://api.twitter.com/2/search/adaptive.json?q=%23AirPollution%20since%3A2020-04-01%20until%3A2020-07-31%20near%3A%22Lagos%2C%20Nigeria%22%20within%3A40mi&count=100&sorted_by_time=true&tweet_mode=extended
[Test URL] https://twitter.com/search?f=tweets&vertical=default&q=%23stopairpollution%20since%3A2020-04-01%20until%3A2020-07-31%20near%3A%22Lagos%2C%20Nigeria%22%20within%3A40mi
https://api.twitter.com/2/search/adaptive.json?q=%23stopairpollution%20since%3A2

In [7]:
final_df.to_csv('during_lockdown_2.csv')

In [8]:
final_df.head()

,id,created_at,original_text,clean_text,polarity,subjectivity,favorite_count,retweet_count,possibly_sensitive,hashtags,user_mentions,place
0,1250214610218688512,Wed Apr 15 00:09:25 +0000 2020,To make the situation worse @IkejaElectric did...,To make situation worse supply windows need op...,0.300000,0.450000,0,0,None,,IkejaElectric,"Ikotun, Nigeria"
1,1250237440629104640,Wed Apr 15 01:40:08 +0000 2020,Kindly help retweet to stop #airpollution thro...,Kindly help retweet stop burning tyres,0.600000,0.900000,0,0,False,airpollution,"Omojuwa, DrOlusesan, ChiefOjukwu","Ikotun, Nigeria"
2,1283785146064416771,Thu Jul 16 15:26:45 +0000 2020,@OstenLaboratory \n\nQuality is our watchword....,Quality watchword We deliver exceptional resul...,0.666667,1.000000,2,1,False,"ostenlaboratory, fieldwork, thursdaymotivation...",OstenLaboratory,"Lagos, Nigeria"
3,1288028180935249920,Tue Jul 28 08:27:03 +0000 2020,Osten Laboratory\nits always something new and...,Osten Laboratoryits always something new excti...,0.318182,0.527273,3,0,False,"airquality, sampecollection, ostenlaboratory, ...",,"Lagos, Nigeria"
4,1285860739245195265,Wed Jul 22 08:54:25 +0000 2020,Air Purifier 🍃\n\nPurifies the air while diffu...,Air Purifier Purifies air diffusing fragrance ...,0.000000,0.000000,0,4,False,"airpurifier, cleanair, perfumemist, home",,"Lagos, Nigeria"
